# Chapitre 1 : Interfaces de programmation d'applications

Une **API** (*Application Programming Interface*), ou une **interface de programmation d’applications**, est un ensemble de classes, fonctions, constantes grâce auquel un programme *fournisseur* met à disposition diverses fonctionnalités pour des programmes *consommateurs*.

C'est le programme fournisseur qui définit de quelle façon les fonctionnalités sont mises à disposition des consommateurs, en imposant éventuellement un certain nombre de contraintes d'accès à l'API (par exemple, la nécessité d'utiliser une clé ou la limitation du nombre d'appels).

## Premier exemple : la _Base Adresse Nationale_

La _Base Adresse Nationale_ est une API gratuite du gouvernement français qui permet d'obtenir un certain nombre d'informations à partir d'une adresse postale.

### Interrogation de l'API

On souhaite, grâce à l'API, obtenir des informations sur l'adresse postale du lycée Louis Armand. Pour interroger l'API, il suffit d'envoyer une requête GET à l'adresse [https://api-adresse.data.gouv.fr/search/?q=173-boulevard-de-strasbourg&postcode=94130](https://api-adresse.data.gouv.fr/search/?q=173-boulevard-de-strasbourg&postcode=94130). On obtient alors la réponse suivante :

```json
{"type": "FeatureCollection",
 "version": "draft",
 "features": [{"type": "Feature",
               "geometry": {"type": "Point",
                            "coordinates": [2.491306, 48.846716]},
               "properties": {"label": "173 Boulevard de Strasbourg 94130 Nogent-sur-Marne",
                              "score": 0.88381,
                              "housenumber": "173",
                              "id": "94052_8560_00173",
                              "name": "173 Boulevard de Strasbourg",
                              "postcode": "94130",
                              "citycode": "94052",
                              "x": 662668.4,
                              "y": 6860861.49,
                              "city": "Nogent-sur-Marne",
                              "context": "94, Val-de-Marne, Île-de-France",
                              "type": "housenumber",
                              "importance": 0.72191,
                              "street": "Boulevard de Strasbourg"}}],
 "attribution": "BAN",
 "licence": "ETALAB-2.0",
 "query": "173-boulevard-de-strasbourg",
 "filters": {"postcode": "94130"},
 "limit": 5}
```

La réponse reçue est au format JSON (_JavaScript Object Notation_). La signification des champs de la réponse est donnée dans la
[documentation](https://geo.api.gouv.fr/adresse) de l'API.

L'interrogation de l'API peut être réalisée grâce au code Python suivant :

In [ ]:
import requests
url = "https://api-adresse.data.gouv.fr/search/?q=173-boulevard-de-strasbourg&postcode=94130"
reponse = requests.get(url)
reponse = reponse.json()

_Affichage du contenu de la variable_ `reponse`

In [ ]:
reponse

La fonction `get` du module `requests` permet d'envoyer une requête GET à l'URL spécifiée et ensuite de récupérer la réponse.

La commande `reponse.json()` extrait de la réponse les données encodées dans le format JSON, qui est un format de données textuelles que l'on peut traiter en Python comme s'il s'agissait d'un dictionnaire.

### Traitement de la réponse envoyée par l'API

Il est possible de définir une fonction qui interroge l'API à partir d'une adresse postale au choix, passée en paramètre.

***Première version : réponse complète***

In [ ]:
def interroger_API_BAN(adresse, code_postal):
    """
    Interroge l'API Base Adresse Nationale.
    - Entrées : adresse (chaîne de caractères), code_postal (entier ou chaîne de caractères)
    - Sortie : reponse (dictionnaire contenant l'ensemble des données renvoyées par l'API)
    Attention : penser à importer le module requests !
    """
    url = f"https://api-adresse.data.gouv.fr/search/?q={adresse}&postcode={code_postal}"
    reponse = requests.get(url)
    reponse = reponse.json()
    return reponse

_Test de la fonction puis affichage du contenu de la variable_ `reponse`

In [ ]:
reponse = interroger_API_BAN('173 boulevard de Strasbourg', 94130)

In [ ]:
reponse

***Seconde version : réponse partielle***

In [ ]:
def interroger_API_BAN(adresse, code_postal):
    """
    Renvoie les coordonnées géographiques d'un lieu après interrogation de l'API Base Adresse Nationale.
    - Entrées : adresse (chaîne de caractères), code_postal (entier ou chaîne de caractères)
    - Sortie : (lat, long) (couple de coordonnées géographiques)
    Attention : penser à importer le module requests !
    """
    url = f"https://api-adresse.data.gouv.fr/search/?q={adresse}&postcode={code_postal}"
    reponse = requests.get(url)
    reponse = reponse.json()
    coord = reponse['features'][0]['geometry']['coordinates']
    long = coord[0]
    lat = coord[1]
    return (lat, long)

L'utilisation d'une f-string pour l'écriture de l'URL permet d'y intégrer simplement et lisiblement les paramètres d'entrée de la fonction.
	
Une fois la réponse de l'API transformée en dictionnaire, il suffit de _naviguer_ pour récupérer les données qui nous intéressent :
	
- `reponse` est un dictionnaire possédant huit clés : `'type'`, `'version'`, `'features'`, `'attribution'`, `'licence'`, `'query'`, `'filters'` et `'limit'`.
- `reponse['features']` est un tableau contenant un ou plusieurs éléments de type dictionnaire : chaque dictionnaire correspond à une adresse potentielle, de la plus probable (indice `0`) à la moins probable.
- `reponse['features'][0]` est un dictionnaire possédant trois clés : `'type'`, `'geometry'` et `'properties'`.
- `reponse['features'][0]['geometry']` est un dictionnaire possédant deux clés : `'type'` et `'coordinates'`.
- `reponse['features'][0]['geometry']['coordinates']` est un tableau contenant deux éléments de type flottant : la longitude (indice `0`) et la latitude (indice `1`) du lieu.

_Test de la fonction puis affichage du contenu de la variable_ `coordonnees`

In [ ]:
coordonnees = interroger_API_BAN('173 boulevard de Strasbourg', 94130)

In [ ]:
coordonnees

_Affichage d'une adresse sur une carte via le module_ `folium`

In [ ]:
import folium

mon_adresse = '173 boulevard de Strasbourg'
mon_code_postal = 94130
coordonnees = interroger_API_BAN(mon_adresse, mon_code_postal)

mon_quartier = folium.Map(location=coordonnees, tiles='OpenStreetMap', zoom_start=15)
folium.Marker(coordonnees, icon=folium.Icon(color='red', icon='home')).add_to(mon_quartier)
mon_quartier # Affichage de la carte sous cette cellule

⚠️ Le module `folium` peut être installé en exécutant la cellule ci-dessous.

In [ ]:
import sys
!{sys.executable} -m pip install folium